#pinkfish-challenge

    1. Buy on the close on the SAME day a new 20 day high is set

In [1]:
%matplotlib inline

Use future imports for python 3.0 forward compatibility

In [2]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

Other imports

In [3]:
import pandas as pd
import numpy as np
import datetime
from talib.abstract import *
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [4]:
symbol = 'SPY'
capital = 100000
start = datetime.datetime(2016, 1, 1)
end = datetime.datetime.now()

Define high low trade periods

In [5]:
period = 20

Define Strategy Class

In [6]:
class Strategy():
    """ strategy """

    def __init__(self, symbol, capital, start, end, period):
        self._symbol = symbol
        self._capital = capital
        self._start = start
        self._end = end
        self._period = period

    def _algo(self):
        """ Algo:
            1. Buy on the close on the SAME day a new 20 day high is set
        """
        
        cash = self._capital
        shares = 0
        start_flag = True

        for i in range(len(self._ts.index)):

            date = self._ts.index[i]
            high = self._ts['high'][i]
            low = self._ts['low'][i]
            close = self._ts['close'][i]
            period_high = self._ts['period_high'][i-1]

            if pd.isnull(period_high) or self._ts.index[i] < self._start:
                continue
            elif start_flag:
                start_flag = False
                # set start and end
                self._start = self._ts.index[i]
                self._end = self._ts.index[-1]

            # buy
            if self._tlog.num_open_trades() == 0:
                if high > period_high:

                    # calculate shares to buy and remaining cash
                    shares, cash = self._tlog.calc_shares(cash, close)

                    # enter buy in trade log
                    self._tlog.enter_trade(date, close, shares)
                    #print("{0} BUY  {1} {2} @ {3:.2f}".format(date, shares, self._symbol, close))

                    # record daily balance
                    self._dbal.append(date, high, low, close, shares, cash, pf.TradeState.OPEN)

                else:
                    # hold
                    self._dbal.append(date, high, low, close, shares, cash, pf.TradeState.HOLD)

            # sell
            elif (i == len(self._ts.index) - 1):

                # enter sell in trade log
                idx = self._tlog.exit_trade(date, close)
                shares = self._tlog.get_log()['qty'][idx]
                #print("{0} SELL {1} {2} @ {3:.2f}".format(date, shares, self._symbol, close))

                # record daily balance
                self._dbal.append(date, high, low, close, shares, cash, pf.TradeState.CLOSE)   
            
                # update cash
                cash = self._tlog.calc_cash(cash, close, shares)
            
                # update shares
                shares = 0

            # hold
            else:
                self._dbal.append(date, high, low, close, shares, cash, pf.TradeState.HOLD)   

    def run(self):
        self._ts = pf.fetch_timeseries(self._symbol)
        self._ts = pf.select_tradeperiod(self._ts, self._start,
                                         self._end, use_adj=False)
        
        # Add technical indicator: X day high
        period_high = pd.rolling_max(self._ts.high, self._period)
        self._ts['period_high'] = period_high               
        
        self._tlog = pf.TradeLog()
        self._dbal = pf.DailyBal()

        self._algo()

    def get_logs(self):
        """ return DataFrames """
        tlog = self._tlog.get_log()
        dbal = self._dbal.get_log()
        return tlog, dbal

Run Strategy

In [7]:
s = Strategy(symbol, capital, start, end, int(period))
s.run()
s._ts['2016-02-01':'2016-03-01']

,open,high,low,close,volume,adj_close,period_high
date,,,,,,,
2016-02-01,192.53,194.58,191.84,193.65,136061600,191.66,201.90
2016-02-02,191.96,191.97,189.54,190.16,182564900,188.21,201.90
2016-02-03,191.41,191.78,187.10,191.30,205054900,189.33,200.06
2016-02-04,190.71,192.75,189.96,191.60,139531800,189.63,197.44
2016-02-05,190.99,191.67,187.20,187.95,180788300,186.02,195.85
2016-02-08,185.77,186.12,182.80,185.42,191526700,183.51,194.86
2016-02-09,183.36,186.94,183.20,185.43,184513100,183.52,194.86
2016-02-10,186.41,188.34,185.12,185.27,148214100,183.37,194.86
2016-02-11,182.34,184.10,181.09,182.86,219058900,180.98,194.58


Retrieve log DataFrames

In [8]:
s.tlog, s.dbal = s.get_logs()

In [9]:
s.tlog.tail(100)

,entry_date,entry_price,long_short,qty,exit_date,exit_price,pl_points,pl_cash,cumul_total
0,2016-02-22,194.78,long,513.00,2016-07-08,212.65,17.87,9167.31,9167.31
